In [2]:
import pandas as pd

import numpy as np
from scipy import sparse
import nltk
from nltk import bigrams    
import scipy.sparse as sps

In [3]:
news = pd.read_csv('data/estadao_noticias_eleicao.csv')

In [4]:
content = news.titulo + " " + news.conteudo
content = content.fillna("")

In [5]:
def co_occurrence_matrix(corpus):
    vocab = set(corpus)
    vocab = list(vocab)
    n = len(vocab)
   
    vocab_to_index = {word:i for i, word in enumerate(vocab)}
    
    bi_grams = list(bigrams(corpus))

    bigram_freq = nltk.FreqDist(bi_grams).most_common(len(bi_grams))

    I=list()
    J=list()
    V=list()
    
    for bigram in bigram_freq:
        current = bigram[0][1]
        previous = bigram[0][0]
        count = bigram[1]

        I.append(vocab_to_index[previous])
        J.append(vocab_to_index[current])
        V.append(count)
        
    co_occurrence_matrix = sparse.coo_matrix((V,(I,J)), shape=(n,n))

    return co_occurrence_matrix, vocab_to_index

In [9]:
tokens_lists = content.apply(lambda text: text.lower().split())

In [10]:
tokens = [token for tokens_list in tokens_lists for token in tokens_list]

In [17]:
matrix, vocab = co_occurrence_matrix(tokens)

# Consult Bigram Frequency

In [12]:
consultable_matrix = matrix.tocsr()


In [18]:
def consult_frequency(w1, w2):
    return(consultable_matrix[vocab[w1],vocab[w2]])

In [45]:
def get_co_ocurrence(word):
    list_of_occurency = consultable_matrix[vocab[word]].getrow(0).toarray()[0]
    indexs, frequency = zip(*sorted(enumerate(list_of_occurency), key=lambda x: x[1], reverse=True))
    return indexs[:5], frequency[:5]

print(get_co_ocurrence('dilma'))


((100219, 12306, 95592, 53102, 26203), (2419, 731, 690, 492, 248))


# Example

In [15]:
w1 = 'poucos'
w2 = 'recursos'
consult_frequency(w1, w2)

3